In [2]:
import os
import xml.etree.ElementTree as ET
import numpy as np
import cv2
import pickle
import copy
import tensorflow as tf

In [18]:
DATA_PATH ='data'
PASCAL_PATH = os.path.join(DATA_PATH, 'pascal_voc')
CACHE_PATH = os.path.join(PASCAL_PATH, 'cache')

In [48]:
class dataset_pascal_voc(object):
    def __init__(self, phase, rebuild=False):
        self.devkil_path = os.path.join(PASCAL_PATH, 'VOCdevkit')
        self.data_path = os.path.join(self.devkil_path, 'VOC2007')
        self.cache_path = CACHE_PATH
        self.batch_size = 20
        self.image_size = 448
        self.cell_size = 7
        self.classes = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
           'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse',
           'motorbike', 'person', 'pottedplant', 'sheep', 'sofa',
           'train', 'tvmonitor']
        self.class_to_ind = dict(zip(self.classes, range(len(self.classes)))) #create dict where keys = labels
        self.flipped = True
        self.phase = phase
        self.rebuild = rebuild
        self.cursor = 0
        self.epoch = 1
        self.gt_labels = None
        self.prepare()
        
    def prepare(self):
        print("In prepare")
        # contains list of dict where each dicts where each dict contain file path, 3D labels & if flipped or not
        labels_got = self.get_labels()
        #print(labels_got[0:10])
        #print(labels_got[0]['label'])
        
    def get_labels(self):
        # getting file containing data
        cache_file = os.path.join(self.cache_path,'pascal_'+self.phase+'_gt_labels.pkl')
        print(self.cache_path)
        if(os.path.isfile(cache_file) and not self.rebuild):
            print("Getting labels from "+ cache_file)
            with open(cache_file, 'rb') as labels_file:
                labels_got = pickle.load(labels_file)
                
            print("original labels length :", len(labels_got))
            return labels_got

In [49]:
pascal_dataset = dataset_pascal_voc('train')

In prepare
data/pascal_voc/cache
Getting labels from data/pascal_voc/cache/pascal_train_gt_labels.pkl
original labels length : 5011
[{'imname': 'data/pascal_voc/VOCdevkit/VOC2007/JPEGImages/000005.jpg', 'label': array([[[  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        ...,
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ]],

       [[  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.    